In [1]:
import json
import glob
import os
import pandas as pd
import numpy as np

with open("adjacency_list_dags/1000genome.json", "r") as f:
    adjacency_list = json.load(f)

counter = 0
edge_index = []
lookup = {}
for l in adjacency_list:
    lookup[l] = counter
    counter+=1

for l in adjacency_list:
    for e in adjacency_list[l]:
        edge_index.append([lookup[l], lookup[e]])

In [ ]:
graphs = []
classes = {"normal": 0}
counter = 1
for d in os.listdir("data"):
    d = d.split("_")[0]
    if d in classes: continue
    classes[d] = counter
    counter += 1

for d in os.listdir("data"):
    for f in glob.glob(os.path.join("data", d, "1000genome*")):
        graph = {"y": classes[d.split("_")[0]], "edge_index": edge_index, "x":[]}
        features = pd.read_csv(f, index_col=[0])
        for l in lookup:
            if l.startswith("create_dir_") or l.startswith("cleanup_"):
                new_l = l.split("-")[0]
            else:
                new_l = l
            job_features = features[features.index.str.startswith(new_l)][['type', 'ready',
                                   'submit', 'execute_start', 'execute_end', 'post_script_start',
                                   'post_script_end', 'wms_delay', 'pre_script_delay', 'queue_delay',
                                   'runtime', 'post_script_delay', 'stage_in_delay', 'stage_out_delay']].values.tolist()[0]
            
            graph['x'].insert(lookup[l], job_features)
        graphs.append(graph)

#graphs should have the input for pytorch geometric